# Reference:
* [MNIST For Machine Learning Beginners With Softmax Regression](https://datascienceplus.com/mnist-for-machine-learning-beginners-with-softmax-regression/)
* [MNIST机器学习入门](http://wiki.jikexueyuan.com/project/tensorflow-zh/tutorials/mnist_beginners.html)

# Purpose:
* MNIST and Softmax regression using machine learning (ML) and TensorFlow.
  * 單純用 MNIST 和 Softmax regression 來訓練模型，並沒有用到 CNN.

# MNIST Data Set
* The downloaded data set is divided into two parts: 60,000 rows of training data `mnist.train` and 10,000 rows of test data `mnist.test`.
* Such segmentation is important. A separate set of test data must be used in the machine learning model design, not for training but to evaluate the performance of the model, making it easier to generalize the design model to other data sets (generalization).
* Each MNIST data unit consists of two parts: a picture containing handwritten digits and a corresponding tag. We set these pictures to “xs” and set these tags to “ys”. Both the training dataset and the test dataset contain xs and ys.
  * For example, the training dataset image is the `mnist.train.images`training dataset label `mnist.train.labels`.
* Each picture contains 28 pixels x 28 pixels for a total of 784 pixels which in turn is represented as a 784 element array.
  * 因為是灰階圖，所以每個 element 的值只為 0 or 1.
  * 在此例中，我們將二維的 28 x 28 matrix 轉成 一維 784 的向量。
    * 這在此例中並無影響，只要轉換的順序是一致的，即可。
    * 但實際上，這樣會丟失了二維的結構訊息，這是不理想的。較優秀的電腦視覺方法中，會有需要用到這些結構訊息。
    * mnist.train.images is shaped as a [60000, 784] tensor (60000 images, each involving a 784 element array).
* For this tutorial, we make the tag data “one-hot vectors.” A one-hot vector is 0 except for one digit.
  * So in this tutorial, the number n will be represented as a 10-dimensional vector with only one digit in the n-th dimension (starting from 0)
  * For example, the label 0 will be represented as ([1,0,0,0,0,0,0,0,0,0,0]).
  * Therefore, mnist.train.labels a [60000, 10] digital matrix.

# Implement a Regression Model
```
x = tf.placeholder("float", [None, 784])
```
  * X Not a specific value, but a placeholder, we enter this value when TensorFlow runs the calculation.
  * We want to be able to input any number of MNIST images, each flattened into 784-dimensional vectors. We use two-dimensional floating-point tensors to represent these graphs. 
  * The shape of this tensor is [None，784 ]. ( None The first dimension here indicates that this tensor can be of any length.)

```
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
```
* Our model also requires weights and offset values, of course, we can use them as a further input (placeholders), but there is a better way TensorFlow to represent them: Variable.
* One Variable represents a modifiable tensor that exists in TensorFlow’s diagram for describing interactive operations. They can be used to calculate input values and can be modified in calculations.
* Here, we all use the zero tensor to initialize the W and b. Because we want to learn W and b value of their initial value can be freely set.
  * 注意，W 的數量是為 784 x 10. 也就是每個 input (784) 對每個 output (10) 的權重。
  * b The shape is [10], so we can add it directly to the output.

```
y = tf.nn.softmax(tf.matmul(x,W) + b)
```
* Now we can implement our model. Only one line of code!
* `tf.matmul(X，W)` expression x multiplication W
* Then add b and enter the tf.nn.softmax function into the function.

# Training The Model
* In order to train our model, we first need to define an indicator to evaluate this model is good. In fact, in machine learning, we usually define indicators to indicate that a model is bad, this indicator is called `cost` or `loss`, and then try to `minimize this indicator`. However, these two methods are the same.
* A very common and very beautiful cost function is cross-entropy.
* In order to calculate the cross-entropy, we first need to add a new placeholder for entering the correct value:  
```
y_ = tf.placeholder("float", [None,10])
```
* Then we can calculate the cross-entropy:
```
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
```
  * First, tf.log calculate y the logarithm of each element.
  * Next, we put y_ each element and tf.log(y_) corresponding elements are multiplied.
  * Finally, tf.reduce_sum the sum of all calculated tensor elements.
  * Not that the cross-entropy here is not only used to measure a single pair of predictions and true values but the sum of the cross entropies of all 100 images. The prediction performance for 100 data points is better than that for a single data point.
* Now that we know what we need our model to do, training it with TensorFlow is very easy. Because TensorFlow has a graph that describes each of your computational units, it can automatically use the `backpropagation algorithm` to effectively determine how your variables affect the cost value you want to minimize. Then, TensorFlow will use your optimization algorithm to constantly modify variables to reduce costs.
```
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
```
  * Here, we require TensorFlow to use a gradient descent algorithm to minimize the cross-entropy at a learning rate of 0.01.
  * What TensorFlow actually does here is that it adds a series of new calculations to the back of the graph that describes your calculations for backpropagation and gradient descent.
  * When running this operation, it uses the gradient descent algorithm to train your model, fine-tune your variables, and continuously reduce costs.
* We have now set up our model. Before running the calculation, we need to add an operation to initialize the variable we created:
```
init = tf.global_variables_initializer()
```
* Now we can Session start our model in one and initialize the variables:
```
sess = tf.Session()
sess.run(init)
```
* Then start training the model, here we let the model cycle training 1000 times!
```
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
```
  * In each step of the cycle, we randomly grab 100 batches of data points in the training data. Then we use these data points as parameters to replace the previous placeholders train_step.

# Evaluate The Model
* What about the performance of our model? Let’s first find out which labels are correct.
```
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
```
  * tf.argmax 是一个非常有用的函數，它能给出某個 tensor 對象在某一維上的其數據最大值所在的索引值。由于標籤向量是由 0,1 组成，因此最大值1所在的索引位置就是類别標籤，比如 tf.argmax(y, 1) 返回的是模型對於任一輸入 x 預測到的標籤值，而 tf.argmax(y_, 1) 代表正確的標籤，我们可以用 tf.equal 來檢測我们的預測是否真實標籤匹配(索引位置一樣表示匹配)。
  * This line of code will give us a set of Boolean values.
```
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
```
  * We can convert Boolean values to floating point numbers and then average them. For example, it [True, False, True, True] will become [1,0,1,1], get after average 0.75.
* Finally, we calculate the correctness of the learned model on the test data set.
```
print sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
```
  * The final result value should be approximately 91%. Is this result okay? Well, it’s not very good.
  * In fact, this result is very poor. This is because we only use a very simple model. However, with some minor improvements, we can get a correct rate of 97%. The best model can even get more than 99.7% accuracy.

# Output
* step = 1000, batch=100, show accuracy with interval=100
```
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.3825
0.8929
0.8835
0.9085
0.9119
0.9138
0.9038
0.914
0.9139
0.915
spend time: 0:00:00.773685
```
* step = 10000, batch=100, show accuracy with interval=1000
```
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.32
0.9077
0.9177
0.9162
0.9146
0.916
0.9222
0.9208
0.9199
0.9203
spend time: 0:00:06.591477
```

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import datetime

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# define placeholder for inputs to network
x = tf.placeholder("float", [None, 784])  # image data
y_ = tf.placeholder("float", [None, 10])  # image label

# define weight, bias and the output = y
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

# define the loss_function = cross_entropy, optimizer = gradient descent (learning rate = 0.01)
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# define the evaluate function = accuracy
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# initial and run the network
time1 = datetime.datetime.now()
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if i % 1000 == 0:
        # show accuracy every 100 time.
        print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
time2 = datetime.datetime.now()
print("spend time:", time2-time1)